In [8]:
import vowpal_wabbit_next as vw
import random
import matplotlib.pyplot as plt
import pandas as pd
import itertools

from typing import List, Tuple

In [9]:
# VW tries to minimize loss/cost, therefore we will pass cost as -reward
USER_LIKED_ARTICLE = -1.0
USER_DISLIKED_ARTICLE = 0.0

In [10]:
def get_cost(context, action):
    '''
    Tom prefers politics in the morning and music in the afternoon, 
    while Anna enjoys sports in the morning and politics later; 
    the dense reward function simulates this in a format the 
    learner recognizes as cost, rewarding article recommendations 
    that match preferences with a simulated "click."
    '''
    if context["user"] == "Tom":
        if context["time_of_day"] == "morning" and action == "politics":
            return USER_LIKED_ARTICLE
        elif context["time_of_day"] == "afternoon" and action == "music":
            return USER_LIKED_ARTICLE
        else:
            return USER_DISLIKED_ARTICLE
    elif context["user"] == "Anna":
        if context["time_of_day"] == "morning" and action == "sports":
            return USER_LIKED_ARTICLE
        elif context["time_of_day"] == "afternoon" and action == "politics":
            return USER_LIKED_ARTICLE
        else:
            return USER_DISLIKED_ARTICLE

In [11]:
def parse_lines(parser: vw.TextFormatParser, input_str: str) -> List[vw.Example]:
    return [parser.parse_line(line) for line in input_str.split("\n")]

In [14]:
# This function modifies (context, action, cost, probability) to VW friendly format
def to_vw_example_format(context, actions, cb_label=None):
    if cb_label is not None:
        chosen_action, cost, prob = cb_label
    example_string = ""
    example_string += "shared |User user={} time_of_day={}\n".format(
        context["user"], context["time_of_day"]
    )
    
    # import pdb; pdb.set_trace()

    for action in actions:
        if cb_label is not None and action == chosen_action:
            example_string += "0:{}:{} ".format(cost, prob)
        example_string += "|Action article={} \n".format(action)
    # Strip the last newline
    return example_string[:-1]

In [ ]:
context = {
    "user": "Tom", 
    "time_of_day": "morning"
}
actions = ["politics", "sports", "music", "food"]

print(to_vw_example_format(context, actions))

> /var/folders/j4/6wq0rt0j1d7d92jfxsxrxh680000gn/T/ipykernel_96990/2238426710.py(11)to_vw_example_format()
      9     import pdb; pdb.set_trace()
     10 
---> 11     for action in actions:
     12         if cb_label is not None and action == chosen_action:
     13             example_string += "0:{}:{} ".format(cost, prob)



ipdb>  print(example_string)


shared |User user=Tom time_of_day=morning



ipdb>  print(cb_label)


None


...

ipdb>  r


--Return--
<ExecutionRes...> result=None>
> /Users/cjw/Development/code/vowpal-wabbit-notebook/venv/lib/python3.11/site-packages/IPython/core/async_helpers.py(131)_pseudo_sync_runner()
    129         coro.send(None)
    130     except StopIteration as exc:
--> 131         return exc.value
    132     else:
    133         # TODO: do not raise but return an execution result with the right info.



ipdb>  r


> /Users/cjw/Development/code/vowpal-wabbit-notebook/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3073)_run_cell()
   3071             self.showtraceback(running_compiled_code=True)
   3072         finally:
-> 3073             return result
   3074 
   3075     def should_run_async(



ipdb>  q
